In [1]:
import numpy as np 
import scipy.linalg as linalg
import matplotlib.pyplot as plt 
import qiskit
from qiskit import Aer
from qh_gates import *
from qh_circuit import QHCircuit
from tn_simu import TNArchitecture, PEPSArchitecture, MPSArchitecture, IBM27Architecture, IBM16Architecture, TNSimulator
from tqdm import tqdm

In [2]:
def rqc2(arch: TNArchitecture, depth, seed=None):
    from copy import deepcopy
    if seed is not None:
        np.random.seed(seed)
        
    nqubits = arch.nnodes
    qc = QHCircuit(nqubits)
    qqc = qiskit.QuantumCircuit(nqubits)

    edges = deepcopy(arch.edges)
    groups = []
    
    while len(edges) > 0:
        current_subset = []
        nodes_in_subset = []
        remaining_edges = []
        for edge in edges:
            u, v = edge
            if u not in nodes_in_subset and v not in nodes_in_subset:
                current_subset.append(edge)
                nodes_in_subset.append(u)
                nodes_in_subset.append(v)
            else:
                remaining_edges.append(edge)
        edges = remaining_edges

        groups.append(current_subset)
    

    qubit_history = [None for _ in range(nqubits)]
    def random_single_qubit(qubit):
        r = np.random.randint(0, 2)
        if qubit_history[qubit] == None:
            qc.t(qubit)
            qqc.t(qubit)
            qubit_history[qubit] = 't' 
        elif qubit_history[qubit] == 't':
            if r == 0:
                qc.rx(qubit, 0.5*np.pi)
                qqc.rx(0.5*np.pi, qubit)
                qubit_history[qubit] = 'rx'
            else:
                qc.ry(qubit, 0.5*np.pi)
                qqc.ry(0.5*np.pi, qubit)
                qubit_history[qubit] = 'ry'
        elif qubit_history[qubit] == 'rx':
            if r == 0:
                qc.ry(qubit, 0.5*np.pi)
                qqc.ry(0.5*np.pi, qubit)  
                qubit_history[qubit] = 'ry'  
            else:
                qc.t(qubit)
                qqc.t(qubit)
                qubit_history[qubit] = 't'  
        elif qubit_history[qubit] == 'ry':
            if r == 0:
                qc.rx(qubit, 0.5*np.pi)
                qqc.rx(0.5*np.pi, qubit)  
                qubit_history[qubit] = 'rx'  
            else:
                qc.t(qubit)
                qqc.t(qubit)
                qubit_history[qubit] = 't'  
        # p1, p2, p3 = np.random.rand(3) * 2 * 0.5*np.pi
        # qc.u(qubit, p1, p2, p3)
        # qqc.u(p1, p2, p3, qubit)

    for qubit in range(nqubits):
        qc.h(qubit)
        qqc.h(qubit)


    d = 0
    this_nodes = []
    while d < depth:
        d += 1
        edges = groups[d%len(groups)]
        # CZ gate
        remaining_nodes = list(range(nqubits))
        last_nodes = this_nodes
        this_nodes = []
        for q1, q2 in edges:
            remaining_nodes.remove(q1)
            remaining_nodes.remove(q2)
            this_nodes += [q1, q2]

            if np.random.rand() < 0.5:
                qc.cz(q1, q2)
                qqc.cz(q1, q2)
            else:
                qc.cz(q2, q1)
                qqc.cz(q2, q1)

            random_single_qubit(q1)
            random_single_qubit(q2)

        # print(f"d={d}; Remain {remaining_nodes}; last {last_nodes}")
        # for qubit in remaining_nodes:
            # if qubit in last_nodes:
                # random_single_qubit(qubit)

        # for qubit in remaining_nodes:
            # random_single_qubit(qubit)
        
        # Single-qubit gate 
        # for qubit in range(nqubits):
            # random_single_qubit(qubit)

    for qubit in range(nqubits):
        qc.h(qubit)
        qqc.h(qubit) 


    return qc, qqc

In [3]:
arch = MPSArchitecture(16)

In [4]:
qc, qqc = rqc2(arch, 8)
qis_simulator = Aer.get_backend('statevector_simulator')


In [5]:
qis_qc_transpiled = qiskit.transpile(qqc, qis_simulator)
ssv = qis_simulator.run(qis_qc_transpiled).result().get_statevector().data
ssv /= np.linalg.norm(ssv)
probs = np.abs(ssv) ** 2
# probs = probs[np.random.randint(0, len(probs), 1000)]

In [6]:
tn_simulator = TNSimulator(arch, qc, 1e-5, max_chi=16)
# tn_simulator.contraction_order = [0,1,2,3,4,5,7,6,8,9,10,11,12,15,13,14]

In [7]:
tn_simulator.simulate(method='qr-svd')

In [8]:
tn_simulator.get_statistics()
tn_simulator.compute_contraction_flop()

Highest degree: 2; Max bond dimension: 16
Max number of parameters:     7232 (113 KiB)
Current number of parameters: 7232 (113 KiB)


338400

In [9]:
sv = tn_simulator.get_all_amplitudes(verbose=True)

100%|██████████| 65536/65536 [00:03<00:00, 17475.95it/s]


In [10]:
np.inner(sv, ssv.conj())

(1.0000000000000129-3.134564005927154e-16j)

In [10]:
measure_probs = np.zeros((tn_simulator.nqubits, 2))
for qubit in range(tn_simulator.nqubits):
    for state in (0,1):
        measure_probs[qubit, state] = tn_simulator.get_measurement_probability(qubit, state)
measure_probs

array([[0.36456297, 0.34098901],
       [0.19084726, 0.51470472],
       [0.3480003 , 0.35755169],
       [0.28553295, 0.42001904],
       [0.42379336, 0.28175863],
       [0.2032168 , 0.50233518],
       [0.36584299, 0.33970899],
       [0.33389446, 0.37165752],
       [0.36090931, 0.34464268],
       [0.52121404, 0.18433795],
       [0.38980796, 0.31574403],
       [0.60835416, 0.09719783],
       [0.30319475, 0.40235724],
       [0.27842642, 0.42712557],
       [0.34602349, 0.35952849],
       [0.20130142, 0.50425056]])